### Creating the 'melt mask' files

- It was necessary to make files that said when the cell in the grid was in the melt season or not
- This dataset takes in the continuous melt onset and continuous freeze onset data from the NSIDC arctic seasonal trends data-set and converts it into an netcdf file with the dataset for each day between april 15 and october 15. If the cell is melting (between cmo and cfo) that day then the value is 1, if not then the value is 0

In [1]:
from scipy.interpolate import griddata
import pyproj as proj
import numpy as np

args = proj.Proj(proj="laea", lat_0=90, lon_0=0, datum="WGS84", units="m")

crs_wgs = proj.Proj(init='epsg:4326')

def regrid(data_in,
           lon_in,
           lat_in,
           lon_out,
           lat_out,
           method='nearest'):

    xout, yout = proj.transform(crs_wgs, args, np.array(lon_out),np.array(lat_out))

    xin, yin = proj.transform(crs_wgs, args, np.array(lon_in),np.array(lat_in))

    output = griddata((xin.ravel(),yin.ravel()),
                    np.array(data_in).ravel(),
                    (xout,yout),
                    method=method)
    
    return(output)

In [3]:
import xarray as xr

EASE_data = xr.open_dataset('/Users/farrerowsleybrown/Desktop/project/code/grids/EASE_grid.nc')
EASE_lon, EASE_lat = EASE_data.lon.data, EASE_data.lat.data

EASE2_data = xr.open_dataset('/Users/farrerowsleybrown/Desktop/project/code/grids/EASE2_grid.nc')
EASE2_lon, EASE2_lat = EASE2_data.lon.data, EASE2_data.lat.data



In [61]:
# Making a function that returns an xarray dataset with a mask for whether that cell is melting or not (1 or 0)

import datetime

dimX = 448
dimY = 304

# Fetch PS lon and lat data
PS_lat = (np.fromfile(#/insert/path/to/pslats/file,dtype='<i4').reshape(dimX,dimY))/100000
PS_lon = (np.fromfile(#/insert/path/to/pslons/file,dtype='<i4').reshape(dimX,dimY))/100000

# Fetch EASE grid lon and lat
EASE_data = xr.open_dataset(#/insert/path/to/example/ease/data/file)
EASE_lat, EASE_lon = EASE_data.latitude.data, EASE_data.longitude.data

def melt_mask(year):
    monset = xr.open_dataset(#/insert/path/to/seasonal/dataset)
    monset['time'] = monset.indexes['time'].to_datetimeindex()
    
    data = monset.sel({'time': datetime.datetime(year,1,1)})
    EASE_mon = regrid(data['CMO'].data,PS_lon,PS_lat,EASE_lon,EASE_lat)
    EASE_fup = regrid(data['CFO'].data,PS_lon,PS_lat,EASE_lon,EASE_lat)
    
    melt_EASE = xr.Dataset()
    melt_EASE = melt_EASE.assign_coords({'latitude': (('columns', 'rows'), EASE_data.latitude),
                                             'longitude': (('columns', 'rows'), EASE_data.longitude)})
    melt_EASE['melt_onset'] = (('columns', 'rows'), EASE_mon)
    melt_EASE['freeze_up'] = (('columns', 'rows'), EASE_fup)
    
    melt_day = melt_EASE['melt_onset']
    melt_day = melt_day.where(melt_day != 0).fillna(104)
    freeze_day = melt_EASE['freeze_up']
    freeze_day = freeze_day.where(freeze_day != 0).fillna(290)
    
    melt_mask = xr.Dataset()
    melt_mask = melt_mask.assign_coords({'latitude': (('columns', 'rows'), EASE_data.latitude),
                                          'longitude': (('columns', 'rows'), EASE_data.longitude),
                                          'dayofyear': range(105,289)})
    
    melt_mask['melting'] = (('dayofyear', 'columns', 'rows'), np.full((184,361,361), 1.))
    
    for i in range(0, 184):
        day = 105 + i
        melt_mask['melting'][i] = melt_mask['melting'][i].where(melt_day <= day)
        melt_mask['melting'][i] = melt_mask['melting'][i].where(freeze_day >= day)
    
    return melt_mask

In [65]:
for year in range(1982, 2018):
    ds = melt_mask(year)
    ds.to_netcdf(# insert/path/to/save/file)
    ds.close()
    print('done ', year)

/Users/farrerowsleybrown/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  from ipykernel import kernelapp as app


done  1982


/Users/farrerowsleybrown/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  from ipykernel import kernelapp as app


done  1983


/Users/farrerowsleybrown/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  from ipykernel import kernelapp as app


...

done  2015


/Users/farrerowsleybrown/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  from ipykernel import kernelapp as app


done  2016


/Users/farrerowsleybrown/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  from ipykernel import kernelapp as app


done  2017
